In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from my_utils import seq_10, DataLoader
from torch_models.utils import seq2label

device = 'cpu'
trans_func = seq2label(device)
train = seq_10(5000)
test = seq_10(500)
train_loader = DataLoader(train, batch_size=16, trans_func=trans_func)
test_loader = DataLoader(test, batch_size=64, trans_func=trans_func)

In [14]:
from torch_models.models import BoV, LSTMLastHidden, LSTMMaxPool, SingleClassifier
from torch_models.utils import get_device

encoder = LSTMLastHidden(embed_size=50, vocab_size=10, bidirectional=True, num_layers=1, device=device)
model = SingleClassifier(encoder=encoder, output_size=2, hidden_size=None,
                         activation='Tanh', dropout=0, freeze_encoder=False, device=device)

In [16]:
from my_utils import Trainer, EvaluatorC
from torch.optim import Adam

optimizer = Adam(model.parameters())

evaluator = EvaluatorC(model, test_loader)

trainer = Trainer(model, train_loader)
trainer.train(optimizer, max_epoch=1,
              evaluator=evaluator, score_monitor=None, show_log=True, hook_func=None)

epoch 0  	loss: 0.15858403441552726	accuracy: 1.0	
